In [1]:
# Calculate the "ground" truth

import numpy as np
import matplotlib.pyplot as plt
from filterpy.kalman import predict, update

# Variables relating to time:
t = 0.0
dt = 5.0
ENDTIME = 200.0
T = np.linspace(t, ENDTIME, int((ENDTIME - t) / dt + 1))
N_steps = len(T)


def actual_position(N):
    velocity = np.asarray([100, 0])
    pos = np.zeros((N, 2))

    for ind in range(1, N_steps):
        t = T[ind]
        pos[ind, :] = pos[ind - 1, :] + dt * velocity
        # Also add turbulence to velocity!
        velocity = velocity + np.random.randn(2)
        if 100 < t < 200:
            velocity = velocity + np.asarray([-5, +5])
    return pos


GROUND_TRUTH = actual_position(N_steps)
plt.figure()
plt.scatter(GROUND_TRUTH[:, 0], GROUND_TRUTH[:, 1], label="Actual trajectory")

ModuleNotFoundError: No module named 'filterpy'

In [ ]:
# Using filterpy we can use the following Python code:

def read_sensor(ind, t):
    std = 500
    R = std * np.eye(2, 2)
    noise = std * np.random.randn(2)
    z = GROUND_TRUTH[ind] + noise
    return (z, R)


def control_action(t, z, R):
    if t >= 100:
        u = np.asarray([-5, +5])
    else:
        u = np.zeros(2)
    return u

# Build matrices and initial state:
N_hidden_dim = 4
N_visible_dim = 2
N_control_dim = 2
# yapf: disable
# Use this state for the hidden variables:
# X = [x, y, v_x, v_y]
# And this state for the visible variables:
# Z = [x,y]

# State transition matrix
F = np.asarray([[0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0]])
# Process uncertainty / noise
Q = np.asarray([[0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0]])

H = np.asarray([[0, 0, 0, 0],
                [0, 0, 0, 0]])
B = np.asarray([[0, 0],
                [0, 0],
                [0, 0],
                [0, 0]])
# yapf: enable

x = np.asarray([0, 0, 100, 0])
P = 500 * np.eye(N_hidden_dim, N_hidden_dim)  # Precision of the hidden state.

# Maintain history for plotting purposes:
hist_t = np.zeros((N_steps, 1))
hist_x = np.zeros((N_steps, N_hidden_dim))
hist_z = np.zeros((N_steps, N_visible_dim))
hist_u = np.zeros((N_steps, N_control_dim))

# Run simulation
for (ind, t) in enumerate(T):
    z, R = read_sensor(ind, t)
    u = control_action(t, z, R)
    x, P = predict(x, P, F, Q, u, B)
    x, P = update(x, P, z, R, H)

    # Append to history
    hist_t[ind] = t
    hist_x[ind, :] = x
    hist_z[ind, :] = z
    hist_u[ind, :] = u

# Now it's time to plot!
plt.figure()
plt.scatter(GROUND_TRUTH[:, 0], GROUND_TRUTH[:, 1], label="Actual trajectory")
plt.scatter(hist_z[:, 0], hist_z[:, 1], label="Measured trajectory")
plt.scatter(hist_x[:, 0], hist_x[:, 1], label="Inferred trajectory")
plt.legend()
plt.plot()